# Pandas

Pandas ens permetrà treballar amb fulls de càlcul (ja siguin xls o csv) i fer operacions sobre taules de dades.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Carreguem unes dades per lectura
df = pd.read_csv('data/dades.csv')
df.head(5)

Pandas, internament, defineix cada columna com a un vector de numpy d'un tipus homogeni:

In [ ]:
df['Nom']

In [ ]:
df['Nota']

Com que cada columna és un vector de numpy, podem fer operacions amb aquest vector.
Quina és la nota mitja de la classe?

In [ ]:
np.min(df['Nota']), df['Nota'].mean(), df['Nota'].max()

## 1. Entenent el que hem vist

De fet, cada columna d'un `DataFrame` (taula de dades) és el que s'anomena una `Series`. Com dèiem, una sèrie és un conjunt homogeni de dades. Per exemple

![alt text](data/dataframe.png "Title")


In [ ]:
serie_1 = pd.Series([1, 2, 3, 4])
serie_1

In [ ]:
serie_2 = pd.Series([1.2, 3.0, 0.0, 6.5])
serie_2

Les sèries es poden unir per fer un `DataFrame`:

In [ ]:
dg = pd.DataFrame(
    {
        'Nota Continua': serie_1, 
         'Nota Reav': serie_2
    }
)
dg

In [ ]:
# Podem canviar el nom de les files (ATENCIÓ, NOM DE FILA != COLUMNA)
dg.index = ['Pablo', 'Maria', 'Antonio', 'Lucia']
dg

In [ ]:
# I dona una pista visual de què indiquen els noms de les files
dg.index.name = 'Nom'
dg

In [ ]:
# I reanomenar columnes
dg.columns = ['Nota Continua', 'Nota Reavaluació']
dg

In [ ]:
# Canvis de tipus
dg['Nota Continua'] = dg['Nota Continua'].astype(float)
dg

## 2. Formes de crear DataFrame's

In [ ]:
pd.DataFrame(
    data=[
        ['Antonio', 2],
        ['Maria', 3],
        ['Asd', 99]
    ],
    columns=['Nom', 'Nota'], index=['DNI_1', 'DNI_2', 'DNI_3'])

In [ ]:
def crear_dades():
    for i in range(10):
        yield 'abcdefghij'[i], i
        
pd.DataFrame(crear_dades())

## 3. Tornem a les dades inicials

Hi ha dues coses rares:

* El professor s'ha equivocat i tenim 2 `Antonio`s, quan realment solament n'hi ha un
* La Laura no va presentar el treball i té un `NaN`

In [ ]:
df

In [ ]:
# Eliminem els duplicats
df_org = df
df = df.drop_duplicates()
df

In [ ]:
# Posem un 0 a la Laura
df.loc[df['Nom'] == 'Laura', 'Nota'] = 0

In [ ]:
df

In [ ]:
df['Nota'].min(), df['Nota'].mean(), df['Nota'].max()

In [ ]:
df.loc[10, 'Nota'] = np.nan
np.mean(df['Nota'])

**Nota**: Pandas vs Numpy

Quina diferencia veieu?

In [ ]:
import numpy as np
np_array = np.asarray([1, 2, np.nan, 3])
np_array.mean()

In [ ]:
pd_series = pd.Series([1, 2, np.nan, 3])
pd_series.mean()

**Guardem les modificacions a disc**

In [ ]:
df.to_csv('dades_mod.csv', index=None)

In [ ]:
pd.read_csv('data/dades.csv', index_col='Nom')

## 4. Més anàlisi de dades

In [ ]:
df.head()

In [ ]:
# Mirem quanta gent a tret la mateixa nota
df.groupby('Nota').count()

In [ ]:
# Creem una nova columna "Reavaluació"
df['Reav'] = 0 # Amb un valor igual per tothom
df.head()

In [ ]:
df['Reav'] = [4.99, 4.98, 4.97, 4.96, np.nan, 5.00, 4.9, 4, np.nan, 2, 1]
df

In [ ]:
# Creem la nota final
has_improved = df['Reav'] > df['Nota']
has_improved.head()

In [ ]:
df.loc[has_improved, :].head()

In [ ]:
df.loc[has_improved,['Nota', 'Reav']]

In [ ]:
df.loc[has_improved, 'Final'] = df['Reav']
df

In [ ]:
df.loc[~has_improved, 'Final'] = df['Nota'] #np.logical_not
df

In [ ]:
# I ara decidim qui ha aprovat
df['Aprovat'] = np.floor(df['Final']) >= 5
df

In [ ]:
print('Han aprovat {} alumnes, un {:.4}%'.format(df['Aprovat'].sum(), df['Aprovat'].sum() * 100 / df.shape[0]))
print('Han suspés {} alumnes, un {:.4}%'.format((~df['Aprovat']).sum(), (~df['Aprovat']).sum() * 100 / df.shape[0]))

**Matricula d'honor**

In [ ]:
df.head()

In [ ]:
nota_maxima = df['Nota'].max()
nota_maxima

In [ ]:
condicio = df['Nota'] == nota_maxima
df.loc[condicio]

In [ ]:
files_alumnes = df.loc[df['Nota'] == nota_maxima]
files_alumnes

In [ ]:
noms_alumnes = df.loc[df['Nota'] == nota_maxima, 'Nom']
noms_alumnes

In [ ]:
# Fem una menció
for nom in noms_alumnes:
    print('Podria ser millor {}'.format(nom))

Podem iterar files també, però compte amb com s'ha de fer!

In [ ]:
dic = {'a': 1, 'b': 2}
for x in dic:
    print(x)

In [ ]:
# NO així
for fila in files_alumnes:
    print(fila)
    
# Hem iterat tots els valors, no files

In [ ]:
# SÍ
for index, fila in files_alumnes.iterrows():
    print(index)
    print(fila)
    print()

In [ ]:
def func(fila):
    print(fila.name)

df.apply(func, axis=1)

In [ ]:
df = df.drop('Nota', axis=1)
df

In [ ]:
dh = df.loc[:, ['Nom','Reav','Final','Aprovat']]
dh

## 5. Combinació de conjunts de dades

### 5.1 Concat

Alguns dels estudis més interessants de dades provenen de la combinació de diferents fonts de dades. Aquestes operacions poden implicar qualsevol cosa, des de la concatenació molt senzilla de dos conjunts de dades diferents, fins a unions i fusions d'estil de base de dades més complicades que gestionen correctament qualsevol superposició entre els conjunts de dades. Les sèries i els DataFrames es creen tenint en compte aquest tipus d'operacions, i Pandas inclou funcions i mètodes que fan que aquest tipus de discussió de dades sigui ràpida i senzilla.

Pandas té una funció, pd.concat(), que té una sintaxi similar a np.concatenate. 

Una diferència important entre np.concatenate i pd.concat és que la concatenació Pandas **conserva els índexs**, fins i tot si el resultat tindrà índexs duplicats!

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

### 5.2 Merge i Join

Una característica essencial que ofereix Pandas són les seves operacions d'unió i fusió d'alt rendiment en memòria. Si alguna vegada heu treballat amb bases de dades, hauríeu d'estar familiaritzat amb aquest tipus d'interacció de dades. La interfície principal per a això és la funció pd.merge, i veurem alguns exemples de com això pot funcionar a la pràctica.

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
pd.merge(df1, df2)

### 5.3 Agregació i agrupació

Una peça essencial d'anàlisi de dades grans és el resum eficient: calculant agregacions com sum(), mean(), median(), min() i max(), en què un sol nombre dóna una visió de la naturalesa d'un potencial gran conjunt de dades. En aquesta secció, explorarem les agregacions a Pandas.

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df.groupby('key')

In [ ]:
df.groupby('key').sum()

# Exercicis

**Exercici 1:** Crea un pd.DataFrame que contingui la informació demanada.

In [ ]:
def crear_dataframe_1(usuari1, usuari2):
    """
    Donada la informació de dos usuaris, `usuari1` i `usuari2`, crea un
    pd.DataFrame que contingui cada un d'aquests usuaris com a una fila.
    La primera fila ha de tenir per índex "99" i la segona "88", de tipus STR.
    Les columnes han de tenir els següents noms:
        "Nom", "Cognom", "Data Registre", "Bitcoin"
        
    :param usuari1: Llista (nativa de python) amb les dades del primer usuari
    :param usuari2: Llista (nativa de python) amb les dades del segon usuari
    :return: DataFrame amb les dades dels usuaris
    """
    # AQUÍ EL TEU CODI
    raise NotImplementedError

In [ ]:
print(crear_dataframe_1(
    ['Mike', 'Strong', '2012-02-03', 99],
    ['Thomas', 'Weak', '2018-01-01', 0.4]
))

In [ ]:
def crear_dataframe_2(x, exponent):
    """
    Donat un vector (np.arrray) i un exponent màxim, crea un 
    DataFrame de pandas on cada columna és la potència
    $x^i$ per cada $i$ entre 0 i `exponent` (incloits). Les columnes han de 
    tenir per nom "x<i>", on <i> és la potència
    
    Per exemple, donat ([1, 2, 3, 4], 2), crearà
    x0 | x1 | x2
    ------------
    1    1    1 
    1    2    4
    1    3    9
    1    4    16
    
    Els indexs de les files són 0, 1, ..., n; on n és el nombre d'elements
    a x
    
    **Pots fer servir 1 sol bucle per iterar de 0 a exponent, cap més**
    
    :param x: np.array unidimensional amb les dades per calcular potències
    :param exponent: enter >= 0, màxim exponent a fer servir
    :return: Un DataFrame de pandas, tal i com s'especifica
    """
    # AQUÍ EL TEU CODI
    raise NotImplementedError

In [ ]:
print(crear_dataframe_2(np.asarray([1, 2, 3, 4]), 5))

**Exercici 2:** Donat un DataFrame, retorna el resultat de les consultes demanades.

In [ ]:
def consultar_basic(df):
    """
    Donat un DataFrame amb noms i notes, retorna solament
    els noms d'aquells usuaris que tinguin un 5 o més
    
    :param df: DataFrame amb dos columnes "Nom", "Nota", amb 1 o més
        files. "Nom" és una string i "Nota" un float
    :return: Un pd.Series o llista/tupla de Pandas amb els noms 
        (i solament els noms) dels alumnes amb 5 o més
    """
    # AQUÍ EL TEU CODI
    raise NotImplementedError

In [ ]:
print(consultar_basic(pd.DataFrame({
    'Nom': ['Antonio', 'Mireia'],
    'Nota': [5.1, 0.1]
})))

In [ ]:
def consultar_dificil(df):
    """
    De totes les files d'un DataFrame, retorna l'índex d'aquella que tingui 
    el menor nombre de NaNs
    
    *Es pot fer sense bucles, consulta la documentació de Pandas, la cheetsheet
    o stackoverflow*
    
    :param df: DataFrame sobre el que operar, les files contenen floats o NaN
    :return: L'índex (int, ja ve donat) de la fila amb menys NaNs
    """
    # AQUÍ EL TEU CODI
    raise NotImplementedError

In [ ]:
print(consultar_dificil(pd.DataFrame([
    [0, np.nan, 3.0, 2, np.nan],
    [np.nan, 1, 2, 3, 4],
    [np.nan, 0, 0, np.nan, np.nan]
])))